In [1]:
import sys
sys.path.append('../src')
from data_processing import load_data, display_data_info, preprocess_data
from feature_engineering import handle_missing_values, filter_outliers_by_room_type, feature_engineering
from evaluation import evaluate_model
from models import get_rf_model, get_xgb_model

/Users/jamesdear/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


1+1

In [2]:
import pandas as pd
from geopy.distance import geodesic
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
def preprocess_data(X,y):

    categorical_features = ['room_type', 'neighbourhood']
    numerical_features = ['availability_365', 'reviews_per_month', 'calculated_host_listings_count', 'distance_to_center']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ],
        remainder='passthrough'  # keep numerical columns as-is
    )

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', get_rf_model())
    ])
    
    return pipeline, X_train, X_test, y_train, y_test

In [3]:
df = load_data('../data/raw/listingss.csv')
display_data_info(df)
df = handle_missing_values(df)
df = filter_outliers_by_room_type(df)
X,Y=feature_engineering(df)
pipeline, X_train, X_test, y_train, y_test = preprocess_data(X, Y)

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69351 entries, 0 to 69350
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              69351 non-null  float64
 1   neighbourhood                   69351 non-null  object 
 2   latitude                        69351 non-null  float64
 3   longitude                       69351 non-null  float64
 4   room_type                       69351 non-null  object 
 5   price                           69351 non-null  int64  
 6   minimum_nights                  69351 non-null  int64  
 7   number_of_reviews               69351 non-null  int64  
 8   last_review                     52571 non-null  object 
 9   reviews_per_month               52571 non-null  float64
 10  calculated_host_listings_count  69351 non-null  int64  
 11  availability_365                69351 non-null  int64  
 12  number_of_reviews_ltm

In [4]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['room_type',
                                                   'neighbourhood'])])),
                ('model',
                 RandomForestRegressor(max_depth=10, n_jobs=-1,
                                       random_state=42))])

In [5]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred = pipeline.predict(X_test)

# Evaluate performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"R² Score: {r2:.2f}")


Mean Absolute Error: 46.28
Mean Squared Error: 4791.97
R² Score: 0.52
